# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [19]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vaini,-21.2000,-175.2000,299.24,94,40,4.12,TO,1679771978
1,1,salalah,17.0151,54.0924,298.56,68,41,1.16,OM,1679772170
2,2,safford,32.8340,-109.7076,289.12,21,0,3.09,US,1679772171
3,3,cidreira,-30.1811,-50.2056,296.94,87,100,6.49,BR,1679772171
4,4,kurchum,48.5722,83.6542,271.94,67,72,0.16,KZ,1679772172


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [49]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
"Lng",
"Lat",
size="Humidity",
color="City",
geo=True,
tiles='OSM')

# Display the map
# YOUR CODE HERE
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
city_data_dff_data_df_dfta_df_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,vaini,-21.2000,-175.2000,299.24,94,40,4.12,TO,1679771978
1,1,salalah,17.0151,54.0924,298.56,68,41,1.16,OM,1679772170
2,2,safford,32.8340,-109.7076,289.12,21,0,3.09,US,1679772171
3,3,cidreira,-30.1811,-50.2056,296.94,87,100,6.49,BR,1679772171
4,4,kurchum,48.5722,83.6542,271.94,67,72,0.16,KZ,1679772172


In [26]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_cleaned_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0) &
                                        (city_data_df["Wind Speed"] < 4.5) &
                                        (city_data_df["Max Temp"] < 300.15) &
                                        (city_data_df["Max Temp"] > 294.15)]

# Drop any rows with null values
city_data_cleaned_df = city_data_cleaned_df.dropna()

# Display sample data
city_data_cleaned_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
116,116,jambusar,22.0500,72.8000,298.06,57,0,3.10,IN,1679771970
131,131,fez,34.0372,-4.9998,295.29,35,0,3.09,MA,1679771958
252,252,zaidpur,26.8333,81.3333,295.19,41,0,2.04,IN,1679772315
313,313,islamkot,24.6997,70.1770,297.52,55,0,4.30,PK,1679772345
319,319,sur,22.5667,59.5289,297.32,65,0,2.56,OM,1679772348
349,349,chhabra,24.6667,76.8333,296.45,33,0,2.96,IN,1679772361
430,430,sarankhola,22.3082,89.7897,296.56,87,0,1.99,BD,1679772406
545,545,marsh harbour,26.5412,-77.0636,298.31,68,0,3.78,BS,1679772469


### Step 3: Create a new DataFrame called `hotel_df`.

In [51]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_cleaned_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
116,jambusar,IN,22.0500,72.8000,57,
131,fez,MA,34.0372,-4.9998,35,
252,zaidpur,IN,26.8333,81.3333,41,
313,islamkot,PK,24.6997,70.1770,55,
319,sur,OM,22.5667,59.5289,65,
349,chhabra,IN,24.6667,76.8333,33,
430,sarankhola,BD,22.3082,89.7897,87,
545,marsh harbour,BS,26.5412,-77.0636,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [74]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "api_key": geoapify_key,
    "radius": radius,
    "type": "hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat}, {lng}"
    params["bias"] = f"proximity:{lat}, {lng}"
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
jambusar - nearest hotel: No hotel found
fez - nearest hotel: No hotel found
zaidpur - nearest hotel: No hotel found
islamkot - nearest hotel: No hotel found
sur - nearest hotel: No hotel found
chhabra - nearest hotel: No hotel found
sarankhola - nearest hotel: No hotel found
marsh harbour - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
116,jambusar,IN,22.0500,72.8000,57,No hotel found
131,fez,MA,34.0372,-4.9998,35,No hotel found
252,zaidpur,IN,26.8333,81.3333,41,No hotel found
313,islamkot,PK,24.6997,70.1770,55,No hotel found
319,sur,OM,22.5667,59.5289,65,No hotel found
349,chhabra,IN,24.6667,76.8333,33,No hotel found
430,sarankhola,BD,22.3082,89.7897,87,No hotel found
545,marsh harbour,BS,26.5412,-77.0636,68,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [75]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points(
"Lng",
"Lat",
color="City",
hover_cols = ["Country", "Hotel Name"],
geo=True,
width=800,
height=500,
tiles='OSM')

# Display the map
map_plot2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)